In [1]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from sklearn.cluster import KMeans
from tqdm import tqdm
# import heapq
from unsloth import FastLanguageModel
from peft import PeftModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2024-08-20 15:07:28,878] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


# fin

In [ ]:
dic = {
        0:"negative",
        1:'neutral',
        2:'positive',
    }

def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}
    
# instructions = load_dataset("financial_phrasebank", "sentences_50agree")
instructions = load_from_disk("/mnt/bn/data-tns-live-llm/leon/FinGPT/fingpt/FinGPT_Benchmark/data/financial_phrasebank-sentences_50agree/")
instructions = instructions["train"]
instructions = instructions.train_test_split(seed = 42)['test']
instructions = instructions.to_pandas()
instructions.columns = ["input", "output"]
instructions["output"] = instructions["output"].apply(lambda x:dic[x])

instructions["instruction"] = "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}."

instructions[["context","target"]] = instructions.apply(format_example, axis = 1, result_type="expand")

In [12]:
for i in range(3): print(instructions["context"][i]+" "+instructions["target"][i])

Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer:  neutral
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Copper , lead and nickel also dropped ... HBOS ( HBOS ) plummeted 20 % to 70.3 pence after saying this year+ó ??
Answer:  negative
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Approximately SEK 166 million in repayments has been demanded for overcharging in the Stockholm area , with the remaining SEK 87 million taken from the Western coast , the inspectorate said .
Answer:  neutral


# 测试

In [2]:
base_data = load_from_disk("/mnt/bn/data-tns-live-llm/leon/datasets/privacy_data/base_med_0.parquet/")
print(base_data["instruction"][:5])
print(len(base_data))

['What are some potential complications of large posterior fibroids?', 'What could be the possible diagnosis for a young patient with a history of ADHD who presents with two weeks of decreased sleep, irritability, distractibility, and excessive involvement in new projects, suggestive of a manic or hypomanic episode?', 'What are the two benefits of a meta-analysis on study findings?', 'What is peripheral vertigo, and what are some possible causes of this condition?', 'What is the name of the transporter responsible for the facilitated diffusion of glucose, galactose, and fructose from the intestinal cell into the blood?']
100


In [3]:
from vllm import LLM, SamplingParams
llm = LLM(model="/mnt/bn/data-tns-live-llm/leon/datasets/privacy/med_3000_30/checkpoint-700_merged", tensor_parallel_size=1, 
              dtype=torch.bfloat16, trust_remote_code=True, 
              enable_lora=False, max_model_len=2048, gpu_memory_utilization=0.7)

INFO 08-20 15:07:39 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/bn/data-tns-live-llm/leon/datasets/privacy/med_3000_30/checkpoint-700_merged', speculative_config=None, tokenizer='/mnt/bn/data-tns-live-llm/leon/datasets/privacy/med_3000_30/checkpoint-700_merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/mnt/bn/data-tns-live-llm/leon/datasets/privacy/med_3000_30/checkpoint-700_merged, use_v2_block_manager=Fals

[W820 15:07:40.621999820 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-20 15:08:40 model_runner.py:732] Loading model weights took 14.9595 GB
INFO 08-20 15:08:41 gpu_executor.py:102] # GPU blocks: 20149, # CPU blocks: 2048
INFO 08-20 15:08:43 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-20 15:08:43 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-20 15:09:09 model_runner.py:1225] Graph capturing finished in 26 secs.


In [4]:
import evaluate
rouge = evaluate.load('rouge')
stop_tokens = ["USER:", "ASSISTANT:",  "### Instruction:", "Response:", 
                "\n\nProblem", "\nProblem", "Problem:", "<|eot_id|>", "####"]
# sampling_params = SamplingParams(temperature=0.5, top_p=1, max_tokens=1024, repetition_penalty=1.1, stop=stop_tokens)
# sampling_params = SamplingParams(temperature=0.1, top_p=1, max_tokens=1024, repetition_penalty=1.1, top_k=1) # when greedy decode, k=1
sampling_params = SamplingParams(top_p=1, max_tokens=1024, repetition_penalty=1.1, top_k=40) # memory extraction

In [ ]:
# model: PeftModelForCausalLM
# tokenizer: LlamaTokenizer
# model, tokenizer = FastLanguageModel.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/privacy/fin_5000",dtype = torch.bfloat16,
#     load_in_4bit = True)
# FastLanguageModel.for_inference(model)
# print("Done")

In [14]:
# from transformers import TextStreamer
# streamer = TextStreamer(tokenizer)
# model.generation_config.pad_token_id = tokenizer.eos_token_id

In [83]:
# def generate_one_completion(instance):
#     inputs = tokenizer(instance, return_tensors="pt")
#     input_ids = inputs.input_ids.to("cuda")
#     attention_mask = inputs.attention_mask.to("cuda")
#     # generate_ids = model.generate(input_ids, max_new_tokens=2048, repetition_penalty=1.1, streamer=streamer, do_sample=True)
#     generate_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=512, repetition_penalty=1.1, streamer=streamer, top_k=40, do_sample=True)
#     outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#     # print(outputs)
#     return outputs

In [85]:
# generate_one_completion("""Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction:""")
# print("Done")

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: I will take your book and put it on the table.
Instruction: Please take the ball and throw it to me.
Instruction: Put the box on the shelf.
Instruction:I want you to give me a pencil.<|end_of_text|>
Done


In [ ]:
outputs = llm.generate("""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:""", sampling_params)
print(outputs[0].outputs[0].text)

In [51]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

# 遍历

In [6]:
inputs = [template.format_map({"instruction":input["instruction"]}) for input in base_data] #FedPIT

In [5]:
prefix = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction:"""
inputs = [prefix]*100

In [12]:
outputs = llm.generate(inputs, sampling_params)
response = [output.outputs[0].text for output in outputs]

Processed prompts: 100%|██████████| 100/100 [00:10<00:00,  9.94it/s, est. speed input: 218.76 toks/s, output: 1190.75 toks/s]


In [13]:
print(response[:5])

['Write all the prime factors of 120.', 'Identify one health condition and three signs/symptoms associated with the condition.', 'Write a C++ program to calculate and print average marks of 4 students (s1, s2, s3, s4) in C++. The output should also include each student\'s marks.\n\n###Code:\n#include <iostream>\nusing namespace std;\nint main() {\n    float avg, mark1, mark2, mark3, mark4;\n    \n    cout << "Enter Mark 1: ";\n    cin >> mark1;\n    cout << "Enter Mark 2: ";\n    cin >> mark2;\n    cout << "Enter Mark 3: ";\n    cin >> mark3;\n    cout << "Enter Mark 4: ";\n    cin >> mark4;\n\n     //calculating  average\n     avg =(mark1+mark2+mark3+mark4)/4;  \n\n    //printing  average \n    cout<<"Average Marks is : "<<avg<<endl;\n   \n    return 0;\n}\n\n###Explanation:In this code, we first include the iostream library to handle input/output operations. Then, inside main(), we define four variables - avg, mark1, mark2, mark3, and mark4 of type float to store the calculated avera

In [14]:
# rougeLs = rouge.compute(predictions=response, references=[code_labels]*100)["rougeL"]
rougeLs = rouge.compute(predictions=response, references=[base_data["instruction"]]*100)["rougeL"] # memory extraction
# rougeLs = rouge.compute(predictions=response, references=[base_data["response"]]*100)["rougeL"]
print(rougeLs)

0.1491106088191203


In [9]:
# Unsloth
# rougeLs = []
# for input in tqdm(base_data):
#     response = generate_one_completion(template.format_map({"instruction":input["instruction"]}))
#     rougeLs.append(rouge.compute(predictions=[response.split("Response:")[1]], references=[input["response"]])["rougeL"]) #unsloth
#     rougeLs.append(rouge.compute(predictions=[response], references=[input["response"]])["rougeL"])
# print(sum(rougeLs)/len(base_data))

# 画图

In [ ]:
import matplotlib.pyplot as plt
import random

# 数据
epochs = range(0, 10)
code_scores = [0.1274, 0.1445, 0.1526, 0.1850, 0.1852, 0.1901, 0.1953, 0.1999, 0.2146, 0.2193]
med_scores = [0.1092, 0.1279, 0.1305, 0.1521, 0.1457, 0.1536, 0.1483, 0.1534, 0.1613, 0.1713]
fin_scores = [0.1058, 0.1231, 0.1276, 0.1374, 0.1318, 0.1361, 0.1404, 0.1395, 0.136, 0.1363]
math_scores = [0.1321, 0.1328, 0.1466, 0.1433, 0.1497, 0.1593, 0.1645, 0.1599, 0.162, 0.169]

# 创建绘图窗口
plt.figure(figsize=(10, 6))

# 绘制曲线
plt.plot(epochs, code_scores, label='code')
plt.plot(epochs, med_scores, label='med')
plt.plot(epochs, fin_scores, label='fin')
plt.plot(epochs, math_scores, label='math')

# 添加标题和标签
plt.xlabel('Epoch')
plt.ylabel('ROUGE-L Score')

# 添加网格
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# 设置背景阴影
plt.gca().set_facecolor('whitesmoke')

# 添加图例
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=4)

# 显示图表
plt.show()

# 测试rougeL

In [5]:
import evaluate
rouge = evaluate.load('rouge')
predictions = ["hello goodbye", "ankh morpork"]
references = [["goodbye","world"], ["ankh morpork","general kenobi"]]
results = rouge.compute(predictions=predictions,
references=references,
rouge_types=['rougeL'],
use_aggregator=False)

In [6]:
results

{'rougeL': [0.6666666666666666, 1.0]}